<a href="https://colab.research.google.com/github/Muhammad-ismail2134/Neural_Networking/blob/main/happysadFacePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

In [2]:
!pip install kaggle

In [3]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d astraszab/facial-expression-dataset-image-folders-fer2013

facial-expression-dataset-image-folders-fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile

#Unzipping training data
local_zip = './facial-expression-dataset-image-folders-fer2013.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./facial-expression-dataset')

In [6]:
import os

# Directory of train angry data
train_angry_dir = os.path.join('./facial-expression-dataset/data/train/0')
train_angry_names = os.listdir(train_angry_dir)
print(f'TRAIN SET HORSES: {train_angry_names[:10]}')


TRAIN SET HORSES: ['263.png', '6573.png', '3815.png', '5403.png', '15068.png', '22072.png', '13210.png', '6426.png', '24250.png', '18161.png']


In [7]:
print(f'total training horse images: {len(os.listdir(train_angry_dir))}')


total training horse images: 3995


In [8]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 8
ncols = 8

# Index for iterating over images
pic_index = 0

In [10]:
import cv2

# Load the image using OpenCV
image = cv2.imread("./facial-expression-dataset/data/train/0/10.png")

# Print the input shape of the image
print("Input shape:", image.shape)

Input shape: (48, 48, 3)


In [11]:


model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(7, activation='softmax')
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 64)         0

In [13]:

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=['accuracy'])

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        './facial-expression-dataset/data/train',  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 300x300
        batch_size=512,
        # Since you use binary_crossentropy loss, you need binary labels
        class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(
        './facial-expression-dataset/data/val',  # This is the source directory for validation images
        target_size=(48, 48),  # All images will be resized to 300x300
        batch_size=32,
        # Since you use binary_crossentropy loss, you need binary labels
        class_mode='categorical')


Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [15]:
class myCallback(tf.keras.callbacks.Callback):
    # Define the method that checks the accuracy at the end of each epoch
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy') is not None and logs.get('accuracy') > 0.995):
            print("\nThe Accuracy has reached 99.5 %,so cancelling training")
            self.model.stop_training = True

        pass


In [16]:
callbacks = myCallback()

history = model.fit(
      train_generator,
      steps_per_epoch=56,
      epochs=15,
      callbacks=[callbacks],
      validation_data = validation_generator,
      validation_steps=30
      )

Epoch 1/15
56/56 [==============================] - 128s 2s/step - loss: 1.7675 - accuracy: 0.2808 - val_loss: 1.6527 - val_accuracy: 0.3656
Epoch 2/15
56/56 [==============================] - 126s 2s/step - loss: 1.5458 - accuracy: 0.4075 - val_loss: 1.4754 - val_accuracy: 0.4365
Epoch 3/15
56/56 [==============================] - 126s 2s/step - loss: 1.4298 - accuracy: 0.4559 - val_loss: 1.3960 - val_accuracy: 0.4677
Epoch 4/15
56/56 [==============================] - 123s 2s/step - loss: 1.3540 - accuracy: 0.4884 - val_loss: 1.3036 - val_accuracy: 0.5198
Epoch 5/15
56/56 [==============================] - 133s 2s/step - loss: 1.2882 - accuracy: 0.5170 - val_loss: 1.2642 - val_accuracy: 0.5271
Epoch 6/15
56/56 [==============================] - 124s 2s/step - loss: 1.2492 - accuracy: 0.5298 - val_loss: 1.2753 - val_accuracy: 0.5198
Epoch 7/15
56/56 [==============================] - 125s 2s/step - loss: 1.1988 - accuracy: 0.5517 - val_loss: 1.2931 - val_accuracy: 0.5073
Epoch 8/15
56

In [18]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  image_name = os.path.basename(path)
  img = load_img(path, target_size=(48, 48))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(f"Name of image : {image_name}\n")
  print(f"Predicted class is : {np.argmax(classes)} and  {np.max(classes) * 100}% \n")

Saving angry.jpeg to angry.jpeg
Saving angry2.jpeg to angry2.jpeg
Saving happy.jpeg to happy (12).jpeg
Saving happy2.jpeg to happy2 (4).jpeg
Saving sad.jpeg to sad (4).jpeg
Saving sad2.jpeg to sad2 (4).jpeg
1/1 [==============================] - 0s 27ms/step
Name of image : angry.jpeg

Predicted class is : 4 and  43.34629476070404% 

1/1 [==============================] - 0s 25ms/step
Name of image : angry2.jpeg

Predicted class is : 3 and  73.99238348007202% 

1/1 [==============================] - 0s 26ms/step
Name of image : happy (12).jpeg

Predicted class is : 3 and  30.293476581573486% 

1/1 [==============================] - 0s 29ms/step
Name of image : happy2 (4).jpeg

Predicted class is : 3 and  96.5560257434845% 

1/1 [==============================] - 0s 29ms/step
Name of image : sad (4).jpeg

Predicted class is : 5 and  52.149540185928345% 

1/1 [==============================] - 0s 27ms/step
Name of image : sad2 (4).jpeg

Predicted class is : 4 and  56.94786310195923% 

